# Calculating Missing Assignments

We are often being asked to find the number of missing assignments that a student has for a given time range. The purpose of this program is to create a file that will have the number of missing assignments for a student for any given time range as well as the sum of missing assignments for each individual school.

In [ ]:
import numpy as np
import pandas as pd

## Queries

__Query for missing assignments__: LIST STU GBU GBS GBA GBK GBR TCH STU.ID STU.NM STU.GR STU.SC GBK.GN GBR.CD GBK.NM GBK.PD TCH.TE GBU.D1 GBU.CSC GBU.CMK GBA.AN GBA.AD GBA.DD GBA.GC GBS.MK GBA.MX IF GBS.MK = " " AND GBS.MX > 0 AND GBA.GC = 1 AND GBA.DD >= GBR.SD AND GBA.DD <= GBR.ED AND GBK.SC = TCH.SC


__Query for Roster__: LIST STU ENR STU.SC STU.RS STU.ID STU.NM STU.FN STU.LN STU.LF STU.GP ENR.YR ENR.SC ENR.ED ENR.LD ENR.GR ENR.ED

In [ ]:
missing_assignments = pd.read_excel(r"C:\Users\derek.castleman\Desktop\MissingAssignments.xlsx")

roster = pd.read_excel(r"C:\Users\derek.castleman\Desktop\StudentRoster.xlsx")

output = "C:\\Users\\derek.castleman\\Desktop\\Missing_Assignments.xlsx"

In [ ]:
missing_assignments

## Roster

Preparing the roster data for the year of interest by selecting the year in which the academic year begins and then dropping any duplicates a student might have for that year.

In [ ]:
roster

In [ ]:
# Have to select the year of interest for the roster
a = int(input('What is the year in which the school year starts (2021, 2022, 2023, etc.):          '))

In [ ]:
roster_filtered = roster[roster['Year'] == a]
roster_filtered

In [ ]:
# Gets rid of any duplicates for a student for a given year
roster = roster_filtered.drop_duplicates(subset=['Student ID'])
roster

## Missing Assignments

The date range of interest will first be selected and then these inputs will be used to narrow down the missing assignments to just this range.

In [ ]:
missing_assignments['Start Date']= pd.to_datetime(missing_assignments['Start Date']) # Changes start date to datetime
missing_assignments

In [ ]:
missing_assignments['Due Date']= pd.to_datetime(missing_assignments['Due Date']) # Changes due date to datetime
missing_assignments

In [ ]:
# Selects the start date for the time range of interest
b = input('What is the start date you are interested in (mm/dd/yyyy):          ')
b

In [ ]:
b = pd.to_datetime(b) # Turn start date to date time
b

In [ ]:
# Selects the end date for time range of interest
c = input('What is the end date you are interested in (mm/dd/yyyy):          ')

In [ ]:
c = pd.to_datetime(c) # Turn end date to date time
c

In [ ]:
# Narrows down the missing assignments to the dates of interest
dates_interested = missing_assignments[(missing_assignments['Due Date'] >=b) & (missing_assignments['Due Date'] <=c)]
dates_interested

## Student Missing Assignments

The students will each be given a value of one for every single assignment that they are missing within the time range then the total number of assignments that are missing for each one will be calculated.

In [ ]:
# Creates a column that gives a one for each missing assignment
dates_interested['Missing Assignments'] = 1
dates_interested

In [ ]:
# Sums up the missing assignments for each student
missing_assignments = dates_interested.groupby(by=['Student ID', 'Student Name', 'Grade',
                                                   'School'])['Missing Assignments'].sum().reset_index()
missing_assignments

In [ ]:
roster

In [ ]:
# Narrows down the columns of the roster to those that are of interest
roster_final = roster[['Student ID', 'Student Name', 'Grade', 'School']].reset_index(drop=True)
roster_final

In [ ]:
# Merges the missing assignment dataframe with the roster
combined = pd.merge(roster_final, missing_assignments, how='left', on=['Student ID', 'Student Name', 
                                                                      'Grade', 'School'])
combined

In [ ]:
# Students who have no missing assignments are given a zero
combined["Missing Assignments"] = combined["Missing Assignments"].fillna(0)
combined

In [ ]:
student_final = combined

## School Missing Assignments

Enrollment will be calculated for each school by giving each student a number one for being enrolled then summing this column and grouing by school.

The missing assignments for each student will also be grouped by school to get a total for missing assignments.

The average missing assignments by student will be calculated by dividing the total missing assignments by the number of students that are enrolled.

In [ ]:
combined_enrollment = pd.merge(roster_final, missing_assignments, how='left', on=['Student ID', 'Student Name', 
                                                                      'Grade', 'School'])
combined_enrollment

In [ ]:
combined_enrollment["Missing Assignments"] = combined_enrollment["Missing Assignments"].fillna(0)
combined_enrollment

In [ ]:
# Students are giving a number one for being enrolled
combined_enrollment['Enrollment'] = 1
combined_enrollment

In [ ]:
# The total number of assignments are summed up for each school
school_missing = combined_enrollment.groupby(by=['School'])['Missing Assignments'].sum().reset_index()
school_missing

In [ ]:
# The total enrollment for each school is summed up
school_enrollment = combined_enrollment.groupby(by=['School'])['Enrollment'].sum().reset_index()
school_enrollment

In [ ]:
# The summed up enrollment and missing assignments dataframes are merged together
school_missing = pd.merge(school_missing, school_enrollment, how='inner', on='School').reset_index(drop=True)
school_missing

In [ ]:
# Average missing assignment is calculated by dividing total missing assignments by enrollment
school_missing['Avg Missing'] = school_missing['Missing Assignments'] / school_missing['Enrollment']
school_missing

In [ ]:
# The average missing assignments is rounded to two decimal places
decimals = 2
school_missing['Avg Missing'] = school_missing['Avg Missing'].apply(lambda x: round(x, decimals))
school_missing

In [ ]:
# Write dataframe to file

writer = pd.ExcelWriter(output)

student_final.to_excel(writer, sheet_name = 'Student Missing', index=False)
school_missing.to_excel(writer, sheet_name='School Missing', index=False)

writer.save()